In [36]:
import json
import boto3
import time
from botocore.exceptions import ClientError
import os

In [37]:
backupVaultName = 'test-1'
startWindowMinutes = 60
completionWindowMinutes = 120
moveToColdStorageAfterDays = 30
deleteAfterDays = moveToColdStorageAfterDays + 90
recoveryPointTagValue = 'on-demand-backup'
kmsKeyARN = 'enter-arn'
backupRoleARN = 'enter-arn'
ddbTableArn = 'enter arn'

In [38]:
backupClient = boto3.client('backup')

In [39]:
def BackupVaultExists():
  backupVaultsList = backupClient.list_backup_vaults().get('BackupVaultList')
  backupVaultNames = [vault.get('BackupVaultName') for vault in backupVaultsList]
  if backupVaultName in backupVaultNames:
    return True
  else:
    return False

In [40]:
def createBackUpVault(backupVaultName, kmsKeyARN):
  response = backupClient.create_backup_vault(
    BackupVaultName = backupVaultName,
    EncryptionKeyArn = kmsKeyARN
)

In [41]:
def backupDdbTable(backupVaultName, ddbTableArn, backupRoleARN, startWindowMinutes, completionWindowMinutes, moveToColdStorageAfterDays, deleteAfterDays, recoveryPointTagValue):
  response = backupClient.start_backup_job(
    BackupVaultName = backupVaultName,
    ResourceArn = ddbTableArn,
    IamRoleArn = backupRoleARN,
    StartWindowMinutes=startWindowMinutes,
    CompleteWindowMinutes=completionWindowMinutes,
    Lifecycle={
        'MoveToColdStorageAfterDays': moveToColdStorageAfterDays,
        'DeleteAfterDays': deleteAfterDays
    },
    RecoveryPointTags={
        'ddb-backup': recoveryPointTagValue
    }
  )
  display(response)

In [42]:
# def backupNeptune(backupVaultName, ddbTableArn, backupRoleARN, startWindowMinutes, completionWindowMinutes, moveToColdStorageAfterDays, deleteAfterDays, recoveryPointTagValue):
#   response = backupClient.start_backup_job(
#     BackupVaultName = backupVaultName,
#     ResourceArn = enter-neptune-arn-here,
#     IamRoleArn = backupRoleARN,
#     IdempotencyToken='string',
#     StartWindowMinutes=startWindowMinutes,
#     CompleteWindowMinutes=completionWindowMinutes,
#     Lifecycle={
#         'MoveToColdStorageAfterDays': moveToColdStorageAfterDays,
#         'DeleteAfterDays': deleteAfterDays
#     },
#     RecoveryPointTags={
#         'ddb-backup': recoveryPointTagValue
#     }
# )

In [43]:
def lambda_handler(event, context):
  if BackupVaultExists() == False:
    createBackUpVault(backupVaultName, kmsKeyARN)
  backupDdbTable(backupVaultName, ddbTableArn, backupRoleARN, startWindowMinutes, completionWindowMinutes, moveToColdStorageAfterDays, deleteAfterDays, recoveryPointTagValue)

In [44]:
lambda_handler('event', 'context')

{'ResponseMetadata': {'RequestId': '991bba44-889f-4b93-a8c4-52bf10f4348a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:11:00 GMT',
   'content-type': 'application/json',
   'content-length': '197',
   'connection': 'keep-alive',
   'x-amzn-requestid': '991bba44-889f-4b93-a8c4-52bf10f4348a'},
  'RetryAttempts': 0},
 'BackupJobId': 'B45CFFE0-84DD-3D20-D928-BEE85E7B0F21',
 'RecoveryPointArn': 'arn:aws:backup:us-east-1:526329662628:recovery-point:f742bcc1-c89a-4dad-bac6-b5489542604b',
 'CreationDate': datetime.datetime(2022, 6, 2, 17, 54, 56, 524000, tzinfo=tzlocal())}